In [1]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip install --upgrade transformers optimum

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install -U bitsandbytes accelerate transformers peft


Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

In [5]:
import torch, gc

# Clear CUDA cache
torch.cuda.empty_cache()

# Run garbage collector
gc.collect()


17

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True  )

model_name = "meta-llama/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=False,
    token=""
)

tokenizer = AutoTokenizer.from_pretrained(model_name, token="")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

**Prepare Model for Training**

In [8]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [9]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 6,740,512,768 || trainable%: 0.0311


In [10]:
model = get_peft_model(model, config)
model.print_trainable_parameters()


trainable params: 2,097,152 || all params: 6,740,512,768 || trainable%: 0.0311


/home/ramaswamybalasunda.v/.local/lib/python3.8/site-packages/peft/mapping.py:172: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'meta-llama/Llama-2-7b-hf' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


**Preparing Training Dataset**

In [11]:
ds = load_dataset("vignesh0007/anime-data")

In [12]:
ds["train"]

Dataset({
    features: ['text'],
    num_rows: 1560
})

**Tokenization**

In [13]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [14]:
def tokenize_function(examples):
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
        padding="max_length"
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs


tokenized_data = ds.map(tokenize_function, batched=True)

In [15]:
tokenized_data["train"]

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1560
})

In [16]:
from transformers import TrainingArguments
lr = 2e-4
batch_size = 4
num_epochs = 5


training_args = TrainingArguments(
    output_dir="anime-llama-ft",
    per_device_train_batch_size=48,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy="epoch",
    save_strategy="epoch",
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",
)


In [17]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    args=training_args,
    data_collator=data_collator,
)

model.config.use_cache = False
trainer.train()
model.config.use_cache = True

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/ramaswamybalasunda.v/.local/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/ramaswamybalasunda.v/.local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/ramaswam

Step,Training Loss
33,1.854200
66,1.778100
99,1.756300


/home/ramaswamybalasunda.v/.local/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/ramaswamybalasunda.v/.local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/ramaswamybalasunda.v/.local/lib/python3.8/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` 

In [7]:
!nvidia-smi


Sat Apr 12 17:38:16 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:C1:00.0 Off |                    0 |
| N/A   33C    P0              70W / 500W |   7282MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!kill -9 54723

In [19]:
from huggingface_hub import notebook_login
notebook_login()

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [21]:
from huggingface_hub import login
from transformers import AutoTokenizer


# Step 2: Set repo name
hf_name = "vignesh0007"
model_id = hf_name + "/" + "Anime-Gen-Llama-2-7B"

# Step 3: Push model and tokenizer
model.push_to_hub(model_id)
tokenizer.push_to_hub(model_id)


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vignesh0007/Anime-Gen-Llama-2-7B/commit/eb1c6a9f4c7d3e2ef37e1ebfb9d655a95197d783', commit_message='Upload tokenizer', commit_description='', oid='eb1c6a9f4c7d3e2ef37e1ebfb9d655a95197d783', pr_url=None, repo_url=RepoUrl('https://huggingface.co/vignesh0007/Anime-Gen-Llama-2-7B', endpoint='https://huggingface.co', repo_type='model', repo_id='vignesh0007/Anime-Gen-Llama-2-7B'), pr_revision=None, pr_num=None)